In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/FlowPS/data"

Посмотрим на работу алгоритма поиска core marker genes на примере датасета GSE18728:

In [3]:
df = pd.read_csv('/content/drive/My Drive/FlowPS/change_data/GSE18728.csv')
df.head()

,Sample ID,Binary response,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACP1,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATBC,AATF,AATK,ABAT,ABCA1,ABCA11P,ABCA12,ABCA13,ABCA17P,...,ZSCAN12P1,ZSCAN16,ZSCAN16-AS1,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZSWIM8-AS1,ZSWIM9,ZUFSP,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,1,0,49.999995,61.694390,55.086681,504.869616,67.487396,49.999995,49.999995,146.767072,49.999995,236.083686,253.209673,49.999995,49.999995,49.999995,49.999995,49.999995,454.152390,133.092735,142.772159,171.477120,374.272787,49.999995,456.641193,482.544625,223.045863,193.626149,204.033559,263.406450,54.268452,59.950555,434.463034,410.711202,268.318052,195.308027,91.130711,49.999995,49.999995,49.999995,...,49.999995,126.655744,57.817748,128.547562,62.353783,49.999995,253.050474,75.359543,49.999995,108.541359,136.029963,127.746733,113.804213,50.507882,60.819565,49.999995,107.782002,140.381914,104.556188,49.999995,78.601437,49.999995,49.999995,771.521270,181.224354,98.664110,150.443103,136.535135,214.907973,466.134270,205.007773,611.971945,67.813458,50.517972,120.282653,49.999995,376.746338,276.170368,151.806116,591.821194
1,2,0,49.999995,59.855861,49.999995,671.684562,105.250518,49.999995,49.999995,95.916933,49.999995,184.830024,796.682764,49.999995,119.756056,49.999995,49.999995,55.540757,257.182459,121.655580,110.372387,237.489951,452.428015,49.999995,418.719655,675.125747,142.864613,191.508050,223.675516,304.674553,132.390170,49.999995,462.098617,201.022262,352.967984,225.185248,191.807123,49.999995,49.999995,52.834949,...,49.999995,263.971889,55.722259,161.487753,89.847537,49.999995,372.029203,70.444167,49.999995,116.246625,278.745075,193.113644,116.775922,170.691739,96.305720,49.999995,53.949140,122.055261,84.144851,49.999995,111.693040,49.999995,74.762815,459.793778,256.861189,99.428781,134.669771,86.760571,240.901685,307.618742,159.447802,242.905168,86.610169,68.681313,140.956679,49.999995,344.793376,190.563866,97.582430,555.481207
2,3,100,49.999995,67.894015,49.999995,578.926458,185.929982,49.999995,49.999995,124.035383,49.999995,195.422039,519.983023,49.999995,49.999995,49.999995,49.999995,62.158595,308.062923,119.044814,128.604920,177.427423,291.820810,49.999995,412.782052,765.101655,168.503592,143.802125,281.408140,295.155800,65.426277,101.638089,493.105834,238.470732,201.218141,229.845177,146.755897,49.999995,49.999995,49.999995,...,49.999995,159.466834,62.033690,155.805247,82.807509,49.999995,227.971998,52.188412,49.999995,98.405013,189.062289,251.001979,110.504144,214.207748,88.766542,49.999995,61.986777,122.460652,86.138034,49.999995,86.280822,49.999995,49.999995,346.764296,182.167809,86.937698,172.035843,101.668119,224.240244,352.369251,259.192546,399.407479,75.411547,57.712798,121.091096,80.493993,375.449825,281.911714,107.253264,371.015336
3,4,0,49.999995,60.299571,49.999995,206.431748,213.271578,49.999995,49.999995,109.254303,49.999995,190.200672,317.272645,49.999995,49.999995,49.999995,49.999995,93.205704,298.331026,128.621718,110.415838,136.858989,480.387486,49.999995,428.606512,962.716655,207.038924,197.883307,305.490367,401.654270,73.742263,66.475776,658.743022,206.077932,153.790493,141.375791,93.220111,49.999995,49.999995,49.999995,...,49.999995,262.819497,60.458248,149.444536,82.983193,49.999995,309.502768,78.923946,49.999995,122.040114,278.989054,160.427954,122.548720,469.706507,97.978024,49.999995,86.439648,226.417066,83.240127,49.999995,91.510727,49.999995,69.328400,452.588323,177.368537,86.373775,94.810224,111.010522,444.354980,563.253925,618.794055,2991.619043,111.530873,87.686745,163.876571,98.570775,282.683518,119.531889,94.494260,644.816684
4,5,100,57.411779,50.953361,56.037994,494.487431,56.876752,49.999995,49.999995,128.460443,49.999995,192.541803,86

Разбиваем данные на признаки и таргет(в данном случае для проверки корректности работы возьмем 100 первых признаков):

In [4]:
X = df.drop(['Sample ID', 'Binary response'], axis=1)
X = X.iloc[:, :100]

y = df['Binary response']

Оценивать влияние каждого признака будем с помощью классического SVM:

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve, auc

model = SVC(gamma='auto', probability = True)

Запускаем гланый цикл кросс-валидации с перебором всех признаков для каждого объекта датасета:

In [9]:
from tqdm import tqdm

final = []
for i in tqdm(range(len(X))):
  res = []
  X2 = X.drop([i])
  y2 = y.drop([i])

  for j in range(len(X2)):
    if i != j:
      X3 = X2.drop(j)
      y3 = y2.drop(j)

      roc_auc_list = []
      for num, k in enumerate(range(X3.shape[1])):

        X_train = X3.iloc[:, k]
        X_train_new = X_train.values.reshape(-1, 1)

        model.fit(X_train_new, y3)
        pred_prob = model.predict_proba(X_train_new)
        predict = pred_prob[:,1]
          
        roc_auc = roc_auc_score(y3, predict)   
        roc_auc_list.append((num, roc_auc)) 

    res.append(roc_auc_list)

  final.append(res)

100%|██████████| 61/61 [14:32<00:00, 14.31s/it]


Отсортируем для каждого датасета без i-го и j-го объекта все признаки по значению roc_auc и выберем TOP 30:

In [14]:
array = []
for i in range(len(final)):
  arr = []
  for j in range(len(res)):
    temp = sorted(final[i][j], key=lambda x: (x[1]), reverse=True)
    arr.append(temp[:30])
  array.append(arr)

Оставим только номера значимых признаков:

In [15]:
mass = []
for i in range(len(array)):
  mass1 = []
  for j in range(len(array[i])):
    mass2 = []
    for k in range(len(array[i][j])):
      mass2.append(array[i][j][k][0])

    mass1.append(mass2)

  mass.append(mass1)

Посмотрим на пересечение всех значимых признаков для каждого i и тем самым получим N наборов core marker features для каждого образца:

In [16]:
core_marker = []
for i in range(len(mass)):
  temp = list(set.intersection(*map(set, mass[i])))
  core_marker.append(temp)

In [19]:
print(len(core_marker))
core_marker

61


[[34],
 [57, 34, 90, 15],
 [34, 90, 15, 20, 26],
 [34, 20, 90, 15],
 [34, 26, 42, 15, 57, 90],
 [34, 20, 90, 15],
 [90, 20, 15],
 [34, 90, 15],
 [57, 34, 20, 15],
 [34, 20, 90, 15],
 [34, 90, 15],
 [34, 42, 90, 15],
 [90],
 [90, 26, 20, 15],
 [34, 90, 15, 20, 26],
 [34, 15, 20, 57, 90],
 [34, 20, 90, 15],
 [34, 20, 90, 15],
 [34, 90, 15],
 [20, 15],
 [34, 20, 90, 15],
 [90, 20, 46, 15],
 [34, 46, 15, 20, 90],
 [34, 20, 90],
 [34, 90, 15],
 [34, 90, 15],
 [34, 20, 90, 15],
 [34, 90, 15],
 [34, 20, 90, 15],
 [34, 20, 90, 15],
 [34, 20, 90, 15],
 [90, 20, 15],
 [34, 20, 90, 15],
 [34, 20, 90, 15],
 [34, 15, 20, 57, 90],
 [34, 20, 90, 15],
 [90, 20, 15],
 [90, 20, 15],
 [34, 15, 20, 57, 90],
 [26, 20, 90, 15],
 [90, 20, 15],
 [42, 15, 20, 54, 90],
 [20, 15],
 [34, 15],
 [34, 20, 90, 15],
 [34, 90, 15],
 [34, 20, 90, 15],
 [34, 90, 15],
 [34, 20, 90, 15],
 [34, 20, 90, 15],
 [90, 15],
 [90, 46, 15],
 [90, 20, 15],
 [34, 15, 20, 57, 90],
 [34, 15, 20, 57, 90],
 [90, 20, 15],
 [90, 20, 15],
 